In [1]:
import os
import open3d as o3d 
from dataset.utils import readData, readPointCloud
import numpy as np
import tqdm

In [2]:
def maty(eul_angles):
    mat = np.eye(3)
    for angle in eul_angles:
        ang,frame = angle
        ang = ang*np.pi/180
        if frame == 'x':
            r = np.array([[1,0,0],[0,np.cos(ang),np.sin(ang)],[0,-np.sin(ang),np.cos(ang)]])
        elif frame == 'y':
            r = np.array([[np.cos(ang),0,-np.sin(ang)],[0,1,0],[np.sin(ang),0,np.cos(ang)]]) 
        elif frame == 'z':
            r = np.array([[np.cos(ang),np.sin(ang),0],[-np.sin(ang),np.cos(ang),0],[0,0,1]]) 
        mat = r@mat
    return mat.T

In [36]:
def pcd_individual(pcloud,bounds,lidar2camera,pose):
    points = []
    for j in range(pcloud.shape[0]):
        if (abs(pcloud[j][0])) > bounds[0] or (abs(pcloud[j][1])) > bounds[1] or (abs(pcloud[j][2])) > bounds[2] :
            continue
        else:
            points.append(pcloud[j][:3])

    points = np.array(points)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    pcd = pcd.voxel_down_sample(voxel_size=0.5)

    pcd = pcd.rotate(lidar2camera,(0,0,0))
    A = np.asarray(pcd.points)

    src = np.ones((4,A.shape[0]))
    src[0:3,:] = np.copy(A.T)

    tmat = pose
    tmat = np.vstack((tmat,[0,0,0,1]))

    src = np.dot(tmat,src)
    pcd.points = o3d.utility.Vector3dVector(src.T[:,:3])
    return pcd

In [38]:
path = 'dataset/01'
pclouds = []
poses = readData('dataset/01.txt')[:77].reshape(-1,3,4)
bounds = np.array([50,50,10])
lidar2camera = maty([(90,'x'),(90,'z')])
for file in tqdm.tqdm(sorted(os.listdir(path)[:1])):
    pcloud_path = path+'/'+file
    try:
        index = int(file.split('.')[0].split('0')[-1])
    except ValueError:
        index = 0
    pcloud = readPointCloud(pcloud_path)
    ps = pcd_individual(pcloud,bounds,lidar2camera,poses[index])
    pclouds.append(ps)
s=pclouds[0]
for i in pclouds[1:]:
    s+= i
o3d.visualization.draw_geometries([s])

100%|██████████| 1/1 [00:01<00:00,  1.23s/it]
